In [35]:
import os
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
from plotly.subplots import make_subplots
import re

In [36]:
def plot_snap(snap_path, config_path):
	from downscale import downscale_3d
	print(f"loading {snap_path}")
	snap = spio.loadmat(snap_path)
	Psi = np.stack([snap["Psi"][0, i] for i in range(3)], axis=0)
	simConfig = spio.loadmat(config_path)["simConfig"]
	print(f"Psi loaded")
	Lbox = float(simConfig["Lbox"][0, 0][0, 0])
	N = int(simConfig["N"][0, 0][0, 0])
	print(f"N={N}, Lbox={Lbox}")
	Rho = np.sum(np.absolute(Psi), axis = 0)
    
	scale_factor = 4
	grid_axis = np.linspace(-Lbox / 2, Lbox / 2, num=N//scale_factor)
	GX, GY, GZ = np.meshgrid(grid_axis, grid_axis, grid_axis)
	Val = downscale_3d(Rho, scale_factor)
	plot = pgo.Volume(
		x = GX.flatten(),
		y = GY.flatten(),
		z = GZ.flatten(),
		value = np.log1p(Val).flatten(),
		opacity = 0.02,
		cmin = 0.0,
		cmax = 3.0,
		surface_count = 128,
		colorscale = "jet"
	)
	return plot

In [37]:
def plot_sim_snaps(sim_path, progress_indices):
	sim_files = os.listdir(sim_path)
	searcher = r"snap-Psi-(\d+)-(\d+\.\d*)\.mat"
	snap_files = [re.search(searcher, fname) for fname in sim_files]
	snap_files = [(int(f.groups()[0]), float(f.groups()[1])) for f in snap_files if not(f is None)]
	snap_files = sorted(snap_files)
	snap_files = [f"snap-Psi-{iter}-{time}.mat" for (iter, time) in snap_files]
	config_path = os.path.join(sim_path, "simConfig.mat")
	
	outs = []
	for pr in progress_indices:
		ind = int(pr * (len(snap_files) - 1))
		snap_file = snap_files[ind]
		snap_path = os.path.join(sim_path, snap_file)
		outs.append(plot_snap(snap_path, config_path))
	return outs

# plot_sim_snaps("../outputs/2022-07-20/8-solitons-random-128-attractive-run-3/", [0, 0.5, 1])
# plot_sim_snaps("../outputs/2022-07-21/8-solitons-random-128-nosi-run-3/", [0, 0.5, 1])
[nosi0, nosi50, nosi100] = plot_sim_snaps("../outputs/2022-07-20/8-solitons-random-128-attractive-run-3/", [0, 0.5, 1])

loading ../outputs/2022-07-20/8-solitons-random-128-attractive-run-3/snap-Psi-0-0.53.mat
Psi loaded
N=128, Lbox=100.0
loading ../outputs/2022-07-20/8-solitons-random-128-attractive-run-3/snap-Psi-2000-1061.74.mat
Psi loaded
N=128, Lbox=100.0
loading ../outputs/2022-07-20/8-solitons-random-128-attractive-run-3/snap-Psi-4000-2122.42.mat
Psi loaded
N=128, Lbox=100.0


In [39]:
i3d = {"is_3d": True}
fig = make_subplots(
	rows = 1,
	cols = 3,
	specs=[
		[i3d, i3d, i3d],
	]
)
fig.update_layout(height = 500, width = 1500)
fig.add_trace(nosi0, 1, 1)
fig.add_trace(nosi50, 1, 2)
fig.add_trace(nosi100, 1, 3)

fig = pgo.Figure(fig)

js = '''
(function() {{
    const gd = document.getElementById('{plot_id}');
    let isUnderRelayout = 0;

    gd.on('plotly_relayout', (e) => {{
        const camera = Object.entries(e).find(([k, v]) => k.startsWith("scene") && k.endsWith("camera"))[1];
        if (camera && isUnderRelayout == 0) {{
            const relayoutKeys = Object.keys(gd.layout).filter(k => k.startsWith("scene"));
            isUnderRelayout = relayoutKeys.length;
            relayoutKeys.forEach(k => {{
                Plotly.relayout(gd, k + ".camera", camera)
                  .then(() => {{
                      isUnderRelayout--;
                  }});
            }});
        }}
    }})
}})();
'''

In [ ]:
from IPython.core.display import display, HTML
from plotly.offline import plot
import plotly

div = plot(fig, include_plotlyjs=False, output_type='div')
div_id = div.split('=')[1].split()[0].replace("'", "").replace('"', '')

div = '<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>' + div + f'<script>{js.format(plot_id=div_id)}</script>'
display(HTML(div))

In [40]:
fig.write_html("plot.html", include_plotlyjs='cdn', post_script=js)